In [6]:
# Import necessary libraries
import spacy
from spacy.training.example import Example
import random
import json

# Load your dataset in JSON format
# Replace 'your_dataset.json' with the path to your JSON dataset
# The JSON should have a list of objects with 'text' and 'label' keys
with open('../dataset2.json', 'r') as file:
    data = json.load(file)
print(data[:3])

[{'text': "COUR DE CASSATION \n Première Présidence\n _______\n \n N/réf à rappeler : Ord n° 31656\n Pourvoi N° : D 22-23.988\n Demanderesse : La Ste Excilys\n représentée par la SCP Bénabent\n Défendeur : Monsieur le procureur général près la cour d'appel de Paris\n \n \n \n ORDONNANCE\n \n de la déléguée du premier président de la Cour de cassation,\n \n \n Vu le pourvoi n°D 22-23.988, formé par la société Excilys le 9 décembre 2022 contre un arrêt (RG: 22/19210) rendu le 29 novembre 2022 par la cour d'appel d'Aix-en-Provence pôle 1 chambre 7 ; \n \n Vu la constitution en demande de la SCP Bénabent pour la société Excilys ;\n \n Vu le mémoire ampliatif déposé le 15 décembre 2022 ;\n \n Vu la requête présentée le 15 décembre 2022 par la société Excilys et tendant à l'application de l'article 1009 du code de procédure civile ;\n \n Vu l'avis présenté par M. le procureur général le 20 décembre 2022 ;\n \n S'agissant d'un litige incident intervenant à l'occasion d'une instance actuelleme

In [14]:
# Create a spaCy training data format
# This assumes binary classification (you can modify it for multi-class)
train_data = []
for entry in data:
    text = entry['text']
    label = "PROTAGONISTS"
    positions = entry["protagonistsPositions"]
    pos_with_label = []
    for pos in positions:
        pos_with_label.append((pos[0],pos[1], label))
    pos_with_label = list(set(pos_with_label))
    ents = {"entities": pos_with_label}
    
    train_data.append((text, ents))





In [15]:

# Shuffle the training data for randomness
random.shuffle(train_data)


In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm


# Create a blank spaCy model
nlp = spacy.blank("fr")

# Add a Named Entity Recognition (NER) component to the pipeline
ner = nlp.add_pipe("ner")


for _, annotations in train_data:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])
optimizer = nlp.begin_training()
# Number of training iterations
n_iter = 3

# Train the model
for itn in range(n_iter):
    random.shuffle(train_data)
    losses = {}

    for text, annotations in tqdm(train_data, desc=f"Iteration {itn+1}/{n_iter}"):
        doc = nlp.make_doc(text)
        # print(doc,annotations)
        example = Example.from_dict(doc, annotations)
        # print("--------------------")
        examples = [example]

        # Update the model with the training examples
        nlp.update(examples, drop=0.5, losses=losses, sgd=optimizer)

    print(f"Iteration {itn+1}/{n_iter} Losses: {losses}")

Iteration 1/3:  13%|█▎        | 126/1000 [00:32<02:59,  4.87it/s]/workspace/.miniconda3/lib/python3.11/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "SOC.
 
 ZB1
 
 
 
 COUR DE CASSATION
 ____________..." with entities "[(724, 733, 'PROTAGONISTS'), (9878, 9886, 'PROTAGO...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
Iteration 1/3:  42%|████▏     | 422/1000 [01:28<01:33,  6.17it/s]/workspace/.miniconda3/lib/python3.11/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "SOC.
 
 CH9
 
 
 
 COUR DE CASSATION
 ____________..." with entities "[(3585, 3591, 'PROTAGONISTS'), (4159, 4166, 'PROTA...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored 

Iteration 1/3 Losses: {'ner': 21264.25115427076}


Iteration 2/3: 100%|██████████| 1000/1000 [03:21<00:00,  4.97it/s]


Iteration 2/3 Losses: {'ner': 3991.3396740506887}


Iteration 3/3:  93%|█████████▎| 927/1000 [03:08<00:13,  5.44it/s]

In [ ]:
nlp.to_disk('./model')

In [ ]:
for text, _ in train_data[:50]:
    doc = nlp(text)
    print(doc.ents)


In [ ]:

# Create the Named Entity Recognizer (NER) component and add it to the pipeline
nlp.add_pipe("ner", last=True)

# Add labels to the NER component
# Replace 'PROTAGONISTS' with the actual label present in your dataset
ner.add_label("PROTAGONISTS")


In [ ]:
# Define training parameters
# You can adjust these parameters based on your dataset and requirements
epochs = 10
learning_rate = 0.001
print(ner)


In [ ]:
from spacy.tokens import DocBin
# Train the spaCy model
for epoch in range(epochs):
    # Shuffle the training data at the beginning of each epoch
    random.shuffle(train_data)

    # Create an Example object for each training example
    examples = []
    for text, annotation in train_data:
        doc = nlp.make_doc(text)
        examples.append(Example.from_dict(doc, annotation))

    # Update the model with the training examples
    nlp.update(examples, drop=0.5, losses={})

    # Print training progress
    print(f"Epoch {epoch + 1}/{epochs} complete")